### MDI : Lesson 5

Peut-on établir un lien entre la densité de médecins par spécialité  et par territoire et la pratique du dépassement d'honoraires ? Est-ce  dans les territoires où la densité est la plus forte que les médecins  pratiquent le moins les dépassement d'honoraires ? Est ce que la densité de certains médecins / praticiens est corrélé à la densité de population pour certaines classes d'ages (bebe/pediatre, personnes agées / infirmiers etc...) ?

C'est un sujet assez ouvert pris du Hackathon "données de santé" de Etalab. Il y a un github qui contient une partie des données dont vous aurez besoin. Vous pouvez compléter ça avec de l'INSEE et des données sur la démographie des médecins


In [1]:
# Import des librairies utiles
%matplotlib notebook
import math                               # import a package
import numpy as np                        # import a package with specified name
from sklearn import linear_model as lm    # import whole module
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV


from os import mkdir, rmdir, path         # import a function
from inspect import getsourcelines        # get function source code
from scipy.stats import t                 # import student distribution

from multiprocessing.dummy import Pool as ThreadPool # Multi thread processing
import random
import string
import time
import pandas as pd
from pandas.plotting import scatter_matrix
import urllib
import zipfile
import matplotlib.pyplot as plt     # for plots
import seaborn as sns               # for plots
import sys

from IPython.display import Image   # insert image in notebook

sns.set_palette("colorblind")
color_blind_list = sns.color_palette("colorblind", 8)

np.set_printoptions(formatter={'float': '{: 0.5g}'.format}) # Predefined format for printed ndarrays

In [88]:
# Récupérons les données relative aux professions de santé
# source : TO DO
file_pro = "Honoraires_totaux_des_professionnels_de_sante_par_departement_en_2016.xls"
sheets = ["Spécialistes","Généralistes et MEP","Dentistes et ODF","Sages-femmes","Auxiliaires médicaux"]
dfs = []
for sheet in sheets:
    df = pd.read_excel(folder+file_pro, sheet_name = sheet, na_values="nc") # Import des différentes sheet
    df.insert(loc=0,column="groupe professionel",value=sheet) # nouvelle colonne avec le nom de la sheet
    df.rename(columns={df.columns[1] :"Spécialités"}, inplace=True) # harmonisation du nom des colonnes (pour fusion)
    df.head()
    dfs.append(df)
df_pro = pd.concat(dfs, axis=0)

reg_num = "(\d*)-.*"
reg_dep = "\d*- (.*)"
#df_pro["Num_Departement"] = df_pro.DEPARTEMENT.str.extract(reg_num)
#df_pro["DEPARTEMENT"] = df_pro.DEPARTEMENT.str.extract(reg_dep)


df_pro.head()
# suprimer les lignes total (col B)
# reset_index?
# a voir si on rajoute une colonne pour France Metro / Outre Mer


,groupe professionel,Spécialités,DEPARTEMENT,EFFECTIFS,HONORAIRES SANS DEPASSEMENT (Euros),DEPASSEMENTS (Euros),FRAIS DE DEPLACEMENT (Euros),TOTAL DES HONORAIRES (Euros)
0,Spécialistes,02- Anesthésie-réanimation chirurgicale,01- Ain,17,4530079.33,2496180.45,0.0,7026259.78
1,Spécialistes,02- Anesthésie-réanimation chirurgicale,02- Aisne,19,4985259.62,373837.57,0.0,5359097.19
2,Spécialistes,02- Anesthésie-réanimation chirurgicale,03- Allier,19,5102901.70,560978.59,0.0,5663880.29
3,Spécialistes,02- Anesthésie-réanimation chirurgicale,04- Alpes-Hte-Provence,4,NaN,NaN,NaN,NaN
4,Spécialistes,02- Anesthésie-réanimation chirurgicale,05- Hautes-Alpes,5,NaN,NaN,NaN,NaN


In [94]:
# test pour connaitre quels sont les colonnes qui contienne la regex
def which_col_contains(df, regex):
    data = []
    for i in range(df.shape[1]):
        try:
            val = df.iloc[:,i].str.contains(regex).any()
        except:
            val = False
        data.append(val)
    return data

total_data = []
for i in range(df_pro.shape[1]):
    try:
        val = df_pro.iloc[:,i].str.contains("(?i)TOTAL").any()
    except:
        val = False
    total_data.append(val)
total_data

[False, True, True, False, False, False, False, False]

In [91]:
total_data = [df_pro.iloc[:,i].str.contains("(?i)TOTAL").any() for i in [0,1,2]]

In [92]:
total_data

[False, True, True]

In [81]:
len(dfs)

5

In [35]:
null_data = df_pro[df_pro.isnull().any(axis=1)]
print(f'There are {null_data.shape[0]} lines with missing values')

There are 3844 lines with missing values


In [69]:
df_pro.dtypes

groupe professionel                     object
Spécialités                             object
DEPARTEMENT                             object
EFFECTIFS                                int64
HONORAIRES SANS DEPASSEMENT (Euros)    float64
DEPASSEMENTS (Euros)                   float64
FRAIS DE DEPLACEMENT (Euros)           float64
TOTAL DES HONORAIRES (Euros)           float64
Num_Departement                         object
dtype: object

In [84]:
df_pro.iloc[:,2]

0                     Ain
1                   Aisne
2                  Allier
3      Alpes-Hte-Provence
4            Hautes-Alpes
5         Alpes-Maritimes
6                 Ardèche
7                Ardennes
8                  Ariège
9                    Aube
10                   Aude
11                Aveyron
12       Bouches-du-Rhône
13               Calvados
14                 Cantal
15               Charente
16      Charente-Maritime
17                   Cher
18                Corrèze
19           Corse-du-Sud
20            Haute-Corse
21              Côte-d'Or
22          Côtes-d'Armor
23                 Creuse
24               Dordogne
25                  Doubs
26                  Drôme
27                   Eure
28           Eure-et-Loir
29              Finistère
              ...        
594          Haute-Savoie
595         Paris (Ville)
596        Seine-Maritime
597        Seine-et-Marne
598              Yvelines
599           Deux-Sèvres
600                 Somme
601         

In [80]:
df_pro.iloc[:,2].str.contains("TOTAL").any()

False

In [78]:
total_data = [df_pro.iloc[:,i].str.contains("(?i)TOTAL").any() for i in [0,1,2,8]]
total_data

[False, True, False, False]

In [37]:
df_pro.where?

In [26]:
pd.read_excel?

In [11]:
# récupération des données relatives à la population
# Prendre 2016 pour être cohérent avec les données des médecins
# Introduire une colonne tranche d'age, homme femme,  (passage colonne -> ligne)
# retirer les colonnes total
folder = "/Users/macbook/Documents/MS_Telecom_BigData/01_enseignement/INFMDI721_KitDataScience/result/Lesson5/"
file_pop = "estim-pop-dep-sexe-aq-1975-2018.xls"
df_pop = pd.read_excel(folder+file_pop, sheet_name=4,header=[3,4])
df_pop.head()

Départements             Départements  Ensemble                        \
                   Unnamed: 0_level_1 0 à 4 ans 5 à 9 ans 10 à 14 ans   
01                                Ain   40444.0   43971.0     43824.0   
02                              Aisne   33467.0   35246.0     36243.0   
03                             Allier   16253.0   18081.0     19312.0   
04            Alpes-de-Haute-Provence    7910.0    8868.0      9646.0   
05                       Hautes-Alpes    7256.0    8137.0      8660.0   

Départements                                                              \
             15 à 19 ans 20 à 24 ans 25 à 29 ans 30 à 34 ans 35 à 39 ans   
01               37879.0     29861.0     35455.0     40786.0     40750.0   
02               32541.0     26762.0     30359.0     32013.0     30619.0   
03               17686.0     15326.0     15433.0     17264.0     17388.0   
04                8576.0      6542.0      7182.0      8242.0      8373.0   
05                7511.0      5905.0      6801.0      7841.0      8093.0   

Départements                ...         Femmes                          \
             40 à 44 ans    ...    55 à 59 ans 60 à 64 ans 65 à 69 ans   
01               47269.0    ...        20223.0     18639.0     16925.0   
02               35981.0    ...        18538.0     18496.0     16504.0   
03               22125.0    ...        12517.0     13522.0     12657.0   
04               10311.0    ...         5994.0      5867.0      6125.0   
05                9582.0    ...         4960.0      5083.0      4673.0   

Départements                                                              \
             70 à 74 ans 75 à 79 ans 80 à 84 ans 85 à 89 ans 90 à 94 ans   
01               11306.0     10000.0      9033.0      6710.0      3568.0   
02                9882.0     10426.0      9942.0      6906.0      3337.0   
03                8788.0      8985.0      8887.0      6380.0      3322.0   
04                4416.0      3926.0      3377.0      2558.0      1435.0   
05                3277.0      3039.0      2728.0      2000.0      1256.0   

Départements                           
             95 ans et plus     Total  
01                    732.0  319895.0  
02                    498.0  276663.0  
03                    645.0  177858.0  
04                    308.0   83087.0  
05                    294.0   72052.0  

[5 rows x 64 columns]

In [12]:
df_pop.shape

(106, 64)